In [135]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import time

In [38]:
index = 'http://anglicismdictionary.ru/Slovar'
origin = 'http://anglicismdictionary.ru'
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

In [4]:
letters = requests.get(index, headers=user_agent)

In [6]:
letters.encoding= 'utf-8'

In [7]:
letters = letters.text

In [9]:
letters = bs(letters, 'lxml')

In [33]:
dict_letters = []
for td in letters.find_all('table')[0].find_all('td'):
    a = td.find_all('a')
    if a:
        dict_letters.append(a[0]['href'])

In [132]:
def get_words(origin, letter):
    doc = requests.get(origin + letter, headers=user_agent)
    doc.encoding = 'utf-8'
    doc_text = doc.text
    html = bs(doc_text, 'lxml')
    html = html.find("div", {"id": "content"})
    
    res = []
    
    word_origin_pattern = re.compile(r'\((.+?)\)')
    info_pattern = re.compile(r'(?<=\)\. )[а-я\. ]+(?=[А-Я])')

    for p in html.find_all('p'):
        article = dict()
        entry = p.find_all('strong')
        if entry:
            entry = [el.get_text().strip(' (').replace('\xa0', '') for el in entry if el]
            word = [el for el in entry if el.isupper()]
            article['word'] = (' '.join(word))

            forms = [el for el in entry if not el.isupper() and el.isalpha()]
            article['forms'] = (', '.join(forms))

            word_origin = re.findall(word_origin_pattern, p.get_text())
            if word_origin:
                article['full_origin'] = word_origin[0].replace('\xa0', '')
            else:
                article['full_origin'] = ''
                


            info = re.findall(info_pattern, p.get_text())
            article['info'] = ' '.join(info).strip(' ')
            article['html'] = p

            res.append(article)
    return res

In [136]:
articles = []
for letter in dict_letters:
    articles.extend(get_words(origin, letter))
    time.sleep(2)

In [138]:
data = pd.DataFrame()

In [142]:
data = data.from_dict(articles)

In [146]:
data = data[['word', 'info', 'forms', 'full_origin', 'html']]

In [158]:
data.loc[data['info'] == '', 'info'] = '-'

In [160]:
data.loc[data['forms'] == '', 'forms'] = '-'

In [162]:
data.loc[data['full_origin'] == '', 'full_origin'] = '-'

In [169]:
data.to_csv("slovar_full.csv", na_rep="-", sep="\t", quotechar="'", encoding='utf-8', index=False)

In [170]:
data_words = data['word']

In [173]:
data_words.to_csv('slovar.csv', sep='\t', index=False, header=False)